In [11]:
# Import packages
import time
import pandas as pd
import numpy as np
import seaborn as sb
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from tqdm import tqdm 
from torch.utils.data import Dataset,DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn import manifold
import os

In [12]:
import os
print (os.environ['CONDA_DEFAULT_ENV'])

base


In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [14]:
!mkdir -p ckpt
!mkdir -p cache

# Load Data

In [15]:
chats = pd.read_csv('data/chats_2021-05.csv', na_values='', keep_default_na=False)
chats.isna().sum()

timestamp          0
body               0
membership         0
isModerator        0
isVerified         0
id                 0
channelId          0
originVideoId      0
originChannelId    0
dtype: int64

In [16]:
delet = pd.read_csv('data/deletion_events.csv')#, usecols=['id', 'retracted'])
delet = delet[delet['retracted'] == 0]

In [17]:
len(delet)

151601

In [18]:
delet.head()

,timestamp,id,retracted,originVideoId,originChannelId
53,2021-02-11T09:54:44.788000+00:00,1cd5dbe14479ed91d6e63a7591f843a1715ff070,0,blSX7M_n9RA,UCl_gCybOJRIgOXw6Qb4qJzQ
62,2021-02-11T10:03:10.305000+00:00,023a77c0bc231728c7b77d781e58fc64371ea96d,0,blSX7M_n9RA,UCl_gCybOJRIgOXw6Qb4qJzQ
113,2021-02-11T10:22:14.158000+00:00,084b4229299fd1f1ba2f75f80ca18bcd081b2feb,0,dAiMpAgmdSo,UC1DCedRgGHBdm81E1llLhOQ
114,2021-02-11T10:22:24.297000+00:00,084b4229299fd1f1ba2f75f80ca18bcd081b2feb,0,dAiMpAgmdSo,UC1DCedRgGHBdm81E1llLhOQ
146,2021-02-11T10:32:40.165000+00:00,0ffc89ef41122a147fe3d6edde10fed1e1fecd2f,0,-UbSxTkVjjU,UC-hM6YJuNYVAmUWxeIr9FeA


In [19]:
# Merge banned data
delet['delete'] = True
chats = pd.merge(chats, delet[['id', 'delete']], how='left')
chats['delete'].fillna(False, inplace=True)

In [20]:
from collections import Counter
Counter(chats['delete'])

Counter({False: 75713528, True: 13120})

In [21]:
chats

,timestamp,body,membership,isModerator,isVerified,id,channelId,originVideoId,originChannelId,delete
0,2021-05-01T00:00:00.112000+00:00,そうじゃないｗ,1 year,0,0,66b9d029a3e93df01b2626a77a5230c71abe5890,606b88eef42cc40a9e055d9af6deaf5e76244c02,S8tYbUIoHM0,UCp-5t9SrOQwXMU7iIjQfARg,False
1,2021-05-01T00:00:00.141000+00:00,いーやバナナだね,2 months,0,0,63cc28b37c760c52156ad2ea8d3e4036a29b19d9,ed808b843c98965376208c6e7aeb12ee122aa9f1,TfRFrbFbE2k,UChUJbHiTVeGrSkTdBzVfNCQ,False
2,2021-05-01T00:00:00.153000+00:00,���,6 months,0,0,60485bda906da346c40671a687884c8c40a012d2,13fce698eba3f1b1617efa6715c618293183b178,S8tYbUIoHM0,UCp-5t9SrOQwXMU7iIjQfARg,False
3,2021-05-01T00:00:00.172000+00:00,大成功でしょ,non-member,0,0,ff9d8bee4c9608299f7c99a286efd19de1cef0ec,116707993d9886a42c32bc5a1a2b9db2e3524e50,S8tYbUIoHM0,UCp-5t9SrOQwXMU7iIjQfARg,False
4,2021-05-01T00:00:00.216000+00:00,大成功やろ！,non-member,0,0,b7b75f473487a932093e1852eafd2e7741c74e28,d8b410fab159d1c7c6aef425312d3d948c26ee8d,S8tYbUIoHM0,UCp-5t9SrOQwXMU7iIjQfARg,False
...,...,...,...,...,...,...,...,...,...,...
75726643,2021-05-31T23:59:59.679000+00:00,Early Shark!,6 months,0,0,c3fbee73ae9db20b7ddb6455c7bfeebfd3e21def,00dcc57ab797acf161a5dceddf359a41d979fc37,lNVQfuc6D4Y,UCoSrY_IQQVpmIRZ9Xf-y93g,False
75726644,2021-05-31T23:59:59.805000+00:00,"""you like jazz?""",6 months,0,0,58efdd0a936bb8afc2e56fc0db4d7e227d614f5d,e323bdfe0c1e04bae4acb9dd7a4729af44a82f77,lNVQfuc6D4Y,UCoSrY_IQQVpmIRZ9Xf-y93g,False
75726645,2021-05-31T23:59:59.832000+00:00,���,6 months,0,0,34d9a82dff9d3b31e4089869ebb100adae561043,d633141e6d0cc0581c5b38d28e6f9de4737ddca4,lNVQfuc6D4Y,UCoSrY_IQQVpmIRZ9Xf-y93g,False
75726646,2021-05-31T23:59:59.872000+00:00,Hellooooo �,6 months,0,0,1c426edd1dbdeea09fbd716e4ba5e078ea794733,f6860156395b32a8da4d76a75884ce8f43f3e623,lNVQfuc6D4Y,UCoSrY_IQQVpmIRZ9Xf-y93g,False


In [22]:
chats[chats['delete']==False].sample(5)

,timestamp,body,membership,isModerator,isVerified,id,channelId,originVideoId,originChannelId,delete
49006238,2021-05-20T18:27:00.058000+00:00,frozen water does,non-member,0,0,e5ca6bb24c69b7d57bf35a9dd5de653fe49241d0,94d80dc7680678bfed7a2a7327af2e1c2f0a7a48,1LcCMBrCC2Y,UCHsx4Hqa-1ORjQTh9TYDhww,False
64171079,2021-05-26T17:06:59.929000+00:00,こうゆう消しゴムあったよね?!角消しみたいなww,non-member,0,0,8c237716bdea699c58d6c3f988aa40805005c874,8bafd558b7fbb279623a2ef7661d2fd2ad980843,WuG_d3b5aKw,UC-hM6YJuNYVAmUWxeIr9FeA,False
70042748,2021-05-29T14:53:24.395000+00:00,顔ww,non-member,0,0,129afc4370047b46dc1ff61ea1707618f5f6bfe4,d8e9772e15bc3c84d9df32039572b848c2b74ad6,bPrvXay0GQY,UC-hM6YJuNYVAmUWxeIr9FeA,False
64910351,2021-05-27T11:07:52.294000+00:00,ごちそうさまでした。,2 years,0,0,91f6b65ccfc5c38e69041078a17f354533577c43,f0193378773a04688eed0c1d48bb90fc54fcdb8b,UFDi4NP5h9g,UCt0clH12Xk1-Ej5PXKGfdPA,False
71029650,2021-05-30T05:18:08.467000+00:00,エッッッ,non-member,0,0,3f8942951f6606d61b05ed230a9bf56e204f7355,7ff6e351db117b2f2d1edc732390a8ecafc8d3a8,Ov6KR0htGYA,UCvzGlP9oQwU--Y0r9id_jnA,False


In [23]:
chats[chats['delete']==True].sample(5)

,timestamp,body,membership,isModerator,isVerified,id,channelId,originVideoId,originChannelId,delete
34065128,2021-05-14T16:02:26.683000+00:00,マジでなんでこっちのリスナーがわざわざ煽るん,non-member,0,0,c1bb68424396ef67c0aa0868b287171fbc849cfa,1868a06456a2d0f89b1b2593df370649cdf140b6,7TlDbZ31Uvk,UCvUc0m317LWTTPZoBQV479A,True
26895844,2021-05-11T09:45:34.213000+00:00,"STAY HALAL AKHI, NO PIG, ITS HARAAAAM, STAY HA...",non-member,0,0,dd5b4efd53c91b2f1e1b143954445eb1dafe2e68,cee0d59e71fa3fa05b6e7c05f674ec1eb35b8f3b,GhEh45cZMHY,UCdYR5Oyz8Q4g0ZmB4PkTD7g,True
70535874,2021-05-29T22:30:56.800000+00:00,LEWD,non-member,0,0,d18ec0244ea8e4275d3cb7f726835db800c01258,30d0736ad9368250ef5c13da86e0475ca428c1ed,VAgn-bhgXeM,UCyl1z3jo3XHR1riLFKG5UAg,True
58617176,2021-05-24T12:35:10.994000+00:00,Jesus,2 months,0,0,e12118cf48f105f92c293c5c7098d8c08b0058d7,e732336c238f45d9267b497bac68be0dbed849ba,UJJONYdwI2c,UCHsx4Hqa-1ORjQTh9TYDhww,True
37379390,2021-05-16T02:29:49.981000+00:00,wth am i watching,non-member,0,0,86d2d9332730a0cfdce45748792738df4cd4f248,922af201e47137e502c380f70596f895a584a808,oRMh4sP4J4s,UCyl1z3jo3XHR1riLFKG5UAg,True


In [24]:
chats.isnull().sum()

timestamp          0
body               0
membership         0
isModerator        0
isVerified         0
id                 0
channelId          0
originVideoId      0
originChannelId    0
delete             0
dtype: int64

# Data Preprocessing and data exploration

### Label

In [25]:
# 轉化成 1 跟 0 的 label
chats['label'] = chats['delete'].apply(lambda x: 1 if x == True else 0)
Counter(chats['label'])

Counter({0: 75713528, 1: 13120})

### 看一下個月份delete data

In [26]:
delet['timestamp'] = pd.to_datetime(delet['timestamp'])
delet['timestamp'].groupby(delet.timestamp.dt.to_period("M")).agg('count')

/home/peggy/env_peggy/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1101: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


timestamp
2021-02    27294
2021-03    74348
2021-04    34370
2021-05    15366
2021-06      201
Freq: M, Name: timestamp, dtype: int64

### Membership

In [27]:
Counter(chats['membership'])

Counter({'1 year': 6541987,
         '2 months': 11227493,
         '6 months': 12556213,
         'non-member': 38108618,
         '2 years': 1000641,
         '1 month': 2877472,
         'less than 1 month': 3414224})

In [28]:
labelencoder = LabelEncoder()
chats["membership"] = labelencoder.fit_transform(chats["membership"])
chats.head(5)

,timestamp,body,membership,isModerator,isVerified,id,channelId,originVideoId,originChannelId,delete,label
0,2021-05-01T00:00:00.112000+00:00,そうじゃないｗ,1,0,0,66b9d029a3e93df01b2626a77a5230c71abe5890,606b88eef42cc40a9e055d9af6deaf5e76244c02,S8tYbUIoHM0,UCp-5t9SrOQwXMU7iIjQfARg,False,0
1,2021-05-01T00:00:00.141000+00:00,いーやバナナだね,2,0,0,63cc28b37c760c52156ad2ea8d3e4036a29b19d9,ed808b843c98965376208c6e7aeb12ee122aa9f1,TfRFrbFbE2k,UChUJbHiTVeGrSkTdBzVfNCQ,False,0
2,2021-05-01T00:00:00.153000+00:00,���,4,0,0,60485bda906da346c40671a687884c8c40a012d2,13fce698eba3f1b1617efa6715c618293183b178,S8tYbUIoHM0,UCp-5t9SrOQwXMU7iIjQfARg,False,0
3,2021-05-01T00:00:00.172000+00:00,大成功でしょ,6,0,0,ff9d8bee4c9608299f7c99a286efd19de1cef0ec,116707993d9886a42c32bc5a1a2b9db2e3524e50,S8tYbUIoHM0,UCp-5t9SrOQwXMU7iIjQfARg,False,0
4,2021-05-01T00:00:00.216000+00:00,大成功やろ！,6,0,0,b7b75f473487a932093e1852eafd2e7741c74e28,d8b410fab159d1c7c6aef425312d3d948c26ee8d,S8tYbUIoHM0,UCp-5t9SrOQwXMU7iIjQfARg,False,0


### isModerator

In [29]:
Counter(chats['isModerator'])

Counter({0: 75587864, 1: 138784})

In [31]:
import plotly.graph_objects as go
labels=['True', 'False']
a = [len(chats[(chats.label == 1) & (chats.isModerator == 1)]), len(chats[(chats.label == 1) & (chats.isModerator == 0)])]
b = [len(chats[(chats.label == 0) & (chats.isModerator == 1)]), len(chats[(chats.label == 0) & (chats.isModerator == 0)])]
print(f'is Moderator, banned: {a[0]}, normal: {b[0]}')
print(f'not Moderator, banned: {a[1]}, normal: {b[1]}')

# fig = go.Figure(data=[
#     go.Bar(name='Banned', x=labels, y=a),
#     go.Bar(name='Normal', x=labels, y=b)
# ])

# # Change the bar mode
# fig.update_layout(barmode='group')
# fig.update_layout(title_text='isModerator')
# fig.show()

is Moderator, banned: 0, normal: 138784
not Moderator, banned: 13120, normal: 75574744


### isVerified

In [32]:
Counter(chats['isVerified'])

Counter({0: 75706841, 1: 19807})

In [33]:
import plotly.graph_objects as go
labels=['True', 'False']
a = [len(chats[(chats.label == 1) & (chats.isVerified == 1)]), len(chats[(chats.label == 1) & (chats.isVerified == 0)])]
b = [len(chats[(chats.label == 0) & (chats.isVerified == 1)]), len(chats[(chats.label == 0) & (chats.isVerified == 0)])]
print(f'is Verified, banned: {a[0]}, normal: {b[0]}')
print(f'not Verified, banned: {a[1]}, normal: {b[1]}')

# fig = go.Figure(data=[
#     go.Bar(name='Banned', x=labels, y=a),
#     go.Bar(name='Normal', x=labels, y=b)
# ])

# # Change the bar mode
# fig.update_layout(barmode='group')
# fig.update_layout(title_text='isVerified')
# fig.show()

is Verified, banned: 10, normal: 19797
not Verified, banned: 13110, normal: 75693731


### Corr

In [34]:
# alpha = ['label', 'membership', 'isModerator', 'isVerified']
# dataframe = chats[['label', 'membership', 'isModerator', 'isVerified']]

# fig = plt.figure(figsize=(8,8))
# ax = fig.add_subplot(111)
# cax = ax.matshow(dataframe.corr(), interpolation='nearest')
# fig.colorbar(cax)

# ax.set_xticklabels(['']+alpha)
# ax.set_yticklabels(['']+alpha)

# plt.show()

# Experiment

## 0. Sample Data
- Hyper parameter: sample_rate (normal / banned)

In [35]:
sample_rate = 5
banned_sample = chats[chats['delete']==True]
print('banned count:', len(banned_sample))
normal_sample = chats[chats['delete']==False].sample((sample_rate*len(banned_sample)))
print('sample normal count:', len(normal_sample))

banned count: 13120
sample normal count: 65600


In [36]:
sample_chats = normal_sample.append(banned_sample, ignore_index=True)
sample_chats

,timestamp,body,membership,isModerator,isVerified,id,channelId,originVideoId,originChannelId,delete,label
0,2021-05-20T13:03:45.901000+00:00,優先順位の選択か,5,0,0,4c51b62ac94b0e5fbc56f7b865683be544ab0aa3,2bb44c5f6a52940b7d848e14d7f8b8fbacb2bf9a,yp19LT9qJe4,UCvzGlP9oQwU--Y0r9id_jnA,False,0
1,2021-05-03T13:44:44.874000+00:00,かっこいい！,4,0,0,2295f4466368143112596cf729a28c2bfde28b57,f13eb309003c4ea08554ec9219cb2206e1ea4bfe,L1ZoyKeNQsg,UCvaTdHTWBGv3MKj3KVqJVCw,False,0
2,2021-05-06T12:30:22.502000+00:00,あんこちゃん尖ってんなーｗ,6,0,0,56a8041e741695fdce1a3631f148feda09a5438f,3cf6acf9ca83725315a60465166e011bd98bf248,FCryv-rWe4c,UC2kyQhzGOB-JPgcQX9OMgEw,False,0
3,2021-05-17T13:32:00.447000+00:00,ｳﾎｯ？,6,0,0,85f3906977cf554bcf1910c97177509cd48658e3,66e4bdd066abf5d4233802e829f57cf87de5a988,1CBcBYTc4xY,UCl-3q6t6zdZwgIsFZELb7Zg,False,0
4,2021-05-01T20:18:33.008000+00:00,�,2,0,0,7730fd712816b8b2af3cb0f113e5d5caa5645dac,800e12fb3474184737033ed157a16560d23482b8,TQKgGOAFl4E,UCqm3BQLlJfvkTsX_hvm0UmA,False,0
...,...,...,...,...,...,...,...,...,...,...,...
78715,2021-05-31T23:13:17.161000+00:00,You got me so bored my insomnia got cured fina...,6,0,0,ec8928aacc085a86b305ac493c03a7e2af541bf3,1af8d348ce055a1bfa883eda3520d20c4f174955,_2jtFfRfGA4,UCyl1z3jo3XHR1riLFKG5UAg,True,1
78716,2021-05-31T23:17:03.082000+00:00,ETERNAL IS MORE THAN BEING ABLE TO AIM AND SHOOT,6,0,0,885a772450171f9bc7c3929d1ad5f09896ba1924,4041027d4d5d50c44e15680e786ccd18dd1947f3,_2jtFfRfGA4,UCyl1z3jo3XHR1riLFKG5UAg,True,1
78717,2021-05-31T23:18:15.719000+00:00,eat me,6,0,0,07b6b100c711ab1a0008b69d3373940da43f3d11,92e2ca9746b0b4970704a6ec4a36aac01c84202c,nnwZu3fUEcw,UCw-jEa3_788VkvM2zHzrDnw,True,1
78718,2021-05-31T23:27:28.888000+00:00,codigo konami,6,0,0,5759b74e5beb20e6508c398ee01f48c371d1e2dd,e9d05788ca511f62603bef7edb260f30ac007de0,nnwZu3fUEcw,UCw-jEa3_788VkvM2zHzrDnw,True,1


# Tuning

In [37]:
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

In [38]:
model_count = 0
output_df = pd.DataFrame(columns = ['batch size', 'step', 'lr', 'grad_norm', 'optimizer', 'Accurcay', 'Precision', 'Recall', 'Auroc', 'F1 score'])
set_seed(0)

### 測試 learning rate

In [39]:
lr_result = []
lr_list = [4e-5, 1e-5, 2e-5, 3e-5, 5e-5]

In [41]:
for lr in lr_list:
    banned_sample = chats[chats['delete']==True]
    print('banned count:', len(banned_sample))
    normal_sample = chats[chats['delete']==False].sample((5*len(banned_sample)),random_state=1)
    print('sample normal count:', len(normal_sample))
    sample_chats = normal_sample.append(banned_sample, ignore_index=True)

    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(sample_chats['body'], sample_chats['label'], test_size=0.2, random_state=42)
    X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

    print("Number of train data: ", len(X_train))
    print("Number of validation data: ", len(X_valid))
    print("Number of test data: ", len(X_test))
    train_df = pd.concat([X_train, y_train], axis=1)
    train_df.columns = ["text", "labels"]
    valid_df = pd.concat([X_valid, y_valid], axis=1)
    valid_df.columns = ["text", "labels"]
    test_df = pd.concat([X_test, y_test], axis=1)
    test_df.columns = ["text", "labels"]
    train_df.head()


    from simpletransformers.classification import ClassificationModel
    output_name = "outputs/outputs_model_lr_"+str(lr)+"/"
    train_args = {
        "output_dir": output_name,
        "cache_dir": "cache/",
        "overwrite_output_dir": True,

        "fp16": False,
        "fp16_opt_level": "O1",
        "max_seq_length": 200,
        "train_batch_size": 32,
        "eval_batch_size": 32,
        "gradient_accumulation_steps": 1,
        "num_train_epochs": 3,
        "weight_decay": 0,
        "learning_rate": lr,
        "adam_epsilon": 1e-8,
        "warmup_ratio": 0.06,
        "warmup_steps": 0,
        "max_grad_norm": 1.0,
        "optimizer": "AdamW",
        "do_lower_case": False,
        "save_model_every_epoch": False,
        "reprocess_input_data": True,
        "n_gpu": 1,
        "silent": False,
        "use_multiprocessing": False,
        "use_early_stopping": True,
        "early_stopping_patience": 3,
        "early_stopping_delta": 0,
        "early_stopping_metric": "eval_loss",
        "early_stopping_metric_minimize": True,
        "manual_seed": None,
        "encoding": None,
    }
    model = ClassificationModel("roberta", "roberta-base", args=train_args)
    model.train_model(train_df, eval_df=valid_df)

    result, model_outputs, wrong_predictions = model.eval_model(test_df)
    acc = (result['tp']+result['tn'])/(result['tp']+result['tn']+result['fp']+result['fn'])
    precision = result['tp']/(result['tp']+result['fp']) 
    recall = result['tp']/(result['tp']+result['fn'])  
    f1 = (2*precision*recall)/(precision+recall)
    print(result)
    print("Accuracy: ", acc)
    print("F1 score: ", f1)
    print('-'*60)
    lr_result.append(result) 

banned count: 13120
sample normal count: 65600
Number of train data:  56678
Number of validation data:  6298
Number of test data:  15744


Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias', 'classifier.out

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

Running Epoch 1 of 3:   0%|          | 0/1772 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/1772 [00:00<?, ?it/s]

  0%|          | 0/15744 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/492 [00:00<?, ?it/s]

{'mcc': 0.7364967716344245, 'tp': 1884, 'tn': 12767, 'fp': 388, 'fn': 705, 'auroc': 0.9432880154452828, 'auprc': 0.8464772422533985, 'eval_loss': 0.21759208931390772}
Accuracy:  0.9305767276422764
F1 score:  0.7751491462662002
------------------------------------------------------------


In [42]:
# lr_result

In [43]:
count = 0
for result in lr_result:
    print(f'----- learning rate {str(lr_list[count])} -----')
    acc =  (result['tp']+result['tn'])/(result['tp']+result['tn']+result['fp']+result['fn'])
    precision = result['tp']/(result['tp']+result['fp']) 
    recall = result['tp']/(result['tp']+result['fn']) 
    f1 = (2*precision*recall)/(precision+recall)
    print('Accuracy: ', acc)
    print('Precision: ', precision)
    print('Recall: ', recall)
    print("F1 score: ", f1)
    print('\n')
    model_count += 1
    output_df.loc[model_count] = [32, 1, lr_list[count], 1, 'AdamW', acc, precision, recall, result['auroc'], f1]
    count += 1

----- learning rate 4e-05 -----
Accuracy:  0.9319740853658537
Precision:  0.8494475138121547
Recall:  0.712630359212051
F1 score:  0.775047258979206


----- learning rate 1e-05 -----
Accuracy:  0.9300685975609756
Precision:  0.8333333333333334
Recall:  0.7184241019698725
F1 score:  0.7716241443683884


----- learning rate 2e-05 -----
Accuracy:  0.9307672764227642
Precision:  0.8341506910387874
Recall:  0.7226728466589417
F1 score:  0.7744205298013245


----- learning rate 3e-05 -----
Accuracy:  0.930322662601626
Precision:  0.8295053003533569
Recall:  0.7253765932792584
F1 score:  0.7739542550999382


----- learning rate 5e-05 -----
Accuracy:  0.9305767276422764
Precision:  0.829225352112676
Recall:  0.727694090382387
F1 score:  0.7751491462662002




In [44]:
output_df

,batch size,step,lr,grad_norm,optimizer,Accurcay,Precision,Recall,Auroc,F1 score
1,32,1,0.00004,1,AdamW,0.931974,0.849448,0.712630,0.941932,0.775047
2,32,1,0.00001,1,AdamW,0.930069,0.833333,0.718424,0.946809,0.771624
3,32,1,0.00002,1,AdamW,0.930767,0.834151,0.722673,0.945276,0.774421
4,32,1,0.00003,1,AdamW,0.930323,0.829505,0.725377,0.944827,0.773954
5,32,1,0.00005,1,AdamW,0.930577,0.829225,0.727694,0.943288,0.775149


### 測試 accumulation step

In [45]:
step_result = []
step_list = [1, 1, 2, 4]
batch_list = [8, 16, 32, 32]

In [46]:
for batch, step in zip(batch_list, step_list):
    banned_sample = chats[chats['delete']==True]
    print('banned count:', len(banned_sample))
    normal_sample = chats[chats['delete']==False].sample((5*len(banned_sample)),random_state=1)
    print('sample normal count:', len(normal_sample))
    sample_chats = normal_sample.append(banned_sample, ignore_index=True)

    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(sample_chats['body'], sample_chats['label'], test_size=0.2, random_state=42)
    X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

    print("Number of train data: ", len(X_train))
    print("Number of validation data: ", len(X_valid))
    print("Number of test data: ", len(X_test))
    train_df = pd.concat([X_train, y_train], axis=1)
    train_df.columns = ["text", "labels"]
    valid_df = pd.concat([X_valid, y_valid], axis=1)
    valid_df.columns = ["text", "labels"]
    test_df = pd.concat([X_test, y_test], axis=1)
    test_df.columns = ["text", "labels"]
    train_df.head()


    from simpletransformers.classification import ClassificationModel
    output_name = "outputs/outputs_model_step_"+str(step)+"_batch_"+str(batch)+"/"
    train_args = {
        "output_dir": output_name,
        "cache_dir": "cache/",
        "overwrite_output_dir": True,

        "fp16": False,
        "fp16_opt_level": "O1",
        "max_seq_length": 200,
        "train_batch_size": batch,
        "eval_batch_size": batch,
        "gradient_accumulation_steps": step,
        "num_train_epochs": 3,
        "weight_decay": 0,
        "learning_rate": 4e-5,
        "adam_epsilon": 1e-8,
        "warmup_ratio": 0.06,
        "warmup_steps": 0,
        "max_grad_norm": 1.0,
        "optimizer": "AdamW",
        "do_lower_case": False,
        "save_model_every_epoch": False,
        "reprocess_input_data": True,
        "n_gpu": 1,
        "silent": False,
        "use_multiprocessing": False,
        "use_early_stopping": True,
        "early_stopping_patience": 3,
        "early_stopping_delta": 0,
        "early_stopping_metric": "eval_loss",
        "early_stopping_metric_minimize": True,
        "manual_seed": None,
        "encoding": None,
    }
    model = ClassificationModel("roberta", "roberta-base", args=train_args)
    model.train_model(train_df, eval_df=valid_df)

    result, model_outputs, wrong_predictions = model.eval_model(test_df)
    acc = (result['tp']+result['tn'])/(result['tp']+result['tn']+result['fp']+result['fn'])
    precision = result['tp']/(result['tp']+result['fp']) 
    recall = result['tp']/(result['tp']+result['fn'])  
    f1 = (2*precision*recall)/(precision+recall)
    print(result)
    print("Accuracy: ", acc)
    print("F1 score: ", f1)
    print('-'*60)
    step_result.append(result) 

banned count: 13120
sample normal count: 65600
Number of train data:  56678
Number of validation data:  6298
Number of test data:  15744


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias', 'classifier.out

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/7085 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/7085 [00:00<?, ?it/s]

...

Running Epoch 1 of 3:   0%|          | 0/1772 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/1772 [00:00<?, ?it/s]

  0%|          | 0/15744 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/492 [00:00<?, ?it/s]

{'mcc': 0.7325892861000086, 'tp': 1900, 'tn': 12728, 'fp': 427, 'fn': 689, 'auroc': 0.9468299426028226, 'auprc': 0.8479806229968972, 'eval_loss': 0.2070049055098609}
Accuracy:  0.9291158536585366
F1 score:  0.7729861676159481
------------------------------------------------------------


In [47]:
# step_result

In [48]:
count = 0
for result in step_result:
    print(f'----- batch size {str(batch_list[count])}, accumulation step {str(step_list[count])} -----')
    acc =  (result['tp']+result['tn'])/(result['tp']+result['tn']+result['fp']+result['fn'])
    precision = result['tp']/(result['tp']+result['fp']) 
    recall = result['tp']/(result['tp']+result['fn']) 
    f1 = (2*precision*recall)/(precision+recall)
    print('Accuracy: ', acc)
    print('Precision: ', precision)
    print('Recall: ', recall)
    print("F1 score: ", f1)
    print('\n')
    model_count += 1
    output_df.loc[model_count] = [batch_list[count], step_list[count], 4e-5, 1, 'AdamW', acc, precision, recall, result['auroc'], f1]
    count += 1

----- batch size 8, accumulation step 1 -----
Accuracy:  0.9190802845528455
Precision:  0.8174794785127958
Recall:  0.6539204325994592
F1 score:  0.7266094420600859


----- batch size 16, accumulation step 1 -----
Accuracy:  0.9279725609756098
Precision:  0.8281461434370772
Recall:  0.709154113557358
F1 score:  0.7640449438202248


----- batch size 32, accumulation step 2 -----
Accuracy:  0.9316565040650406
Precision:  0.8334067871308947
Recall:  0.7303978370027038
F1 score:  0.7785096747632771


----- batch size 32, accumulation step 4 -----
Accuracy:  0.9291158536585366
Precision:  0.8165019338203696
Recall:  0.7338740826573967
F1 score:  0.7729861676159481




In [49]:
output_df

,batch size,step,lr,grad_norm,optimizer,Accurcay,Precision,Recall,Auroc,F1 score
1,32,1,0.00004,1,AdamW,0.931974,0.849448,0.712630,0.941932,0.775047
2,32,1,0.00001,1,AdamW,0.930069,0.833333,0.718424,0.946809,0.771624
3,32,1,0.00002,1,AdamW,0.930767,0.834151,0.722673,0.945276,0.774421
4,32,1,0.00003,1,AdamW,0.930323,0.829505,0.725377,0.944827,0.773954
5,32,1,0.00005,1,AdamW,0.930577,0.829225,0.727694,0.943288,0.775149
6,8,1,0.00004,1,AdamW,0.919080,0.817479,0.653920,0.875916,0.726609
7,16,1,0.00004,1,AdamW,0.927973,0.828146,0.709154,0.941804,0.764045
8,32,2,0.00004,1,AdamW,0.931657,0.833407,0.730398,0.944792,0.778510
9,32,4,0.00004,1,AdamW,0.929116,0.816502,0.733874,0.946830,0.772986


### 測試 gradient clipping

In [50]:
grad_norm_result = []
grad_norm_list = [0.05, 0.5, 1.5]

In [51]:
for grad_norm in grad_norm_list:
    banned_sample = chats[chats['delete']==True]
    print('banned count:', len(banned_sample))
    normal_sample = chats[chats['delete']==False].sample((5*len(banned_sample)),random_state=1)
    print('sample normal count:', len(normal_sample))
    sample_chats = normal_sample.append(banned_sample, ignore_index=True)

    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(sample_chats['body'], sample_chats['label'], test_size=0.2, random_state=42)
    X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

    print("Number of train data: ", len(X_train))
    print("Number of validation data: ", len(X_valid))
    print("Number of test data: ", len(X_test))
    train_df = pd.concat([X_train, y_train], axis=1)
    train_df.columns = ["text", "labels"]
    valid_df = pd.concat([X_valid, y_valid], axis=1)
    valid_df.columns = ["text", "labels"]
    test_df = pd.concat([X_test, y_test], axis=1)
    test_df.columns = ["text", "labels"]
    train_df.head()


    from simpletransformers.classification import ClassificationModel
    output_name = "outputs/outputs_model_grad_norm_"+str(grad_norm)+"/"
    train_args = {
        "output_dir": output_name,
        "cache_dir": "cache/",
        "overwrite_output_dir": True,

        "fp16": False,
        "fp16_opt_level": "O1",
        "max_seq_length": 200,
        "train_batch_size": 32,
        "eval_batch_size": 32,
        "gradient_accumulation_steps": 1,
        "num_train_epochs": 3,
        "weight_decay": 0,
        "learning_rate": 4e-5,
        "adam_epsilon": 1e-8,
        "warmup_ratio": 0.06,
        "warmup_steps": 0,
        "max_grad_norm": grad_norm,
        "optimizer": "AdamW",
        "do_lower_case": False,
        "save_model_every_epoch": False,
        "reprocess_input_data": True,
        "n_gpu": 1,
        "silent": False,
        "use_multiprocessing": False,
        "use_early_stopping": True,
        "early_stopping_patience": 3,
        "early_stopping_delta": 0,
        "early_stopping_metric": "eval_loss",
        "early_stopping_metric_minimize": True,
        "manual_seed": None,
        "encoding": None,
    }
    model = ClassificationModel("roberta", "roberta-base", args=train_args)
    model.train_model(train_df, eval_df=valid_df)

    result, model_outputs, wrong_predictions = model.eval_model(test_df)
    acc = (result['tp']+result['tn'])/(result['tp']+result['tn']+result['fp']+result['fn'])
    precision = result['tp']/(result['tp']+result['fp']) 
    recall = result['tp']/(result['tp']+result['fn'])  
    f1 = (2*precision*recall)/(precision+recall)
    print(result)
    print("Accuracy: ", acc)
    print("F1 score: ", f1)
    print('-'*60)
    grad_norm_result.append(result) 

banned count: 13120
sample normal count: 65600
Number of train data:  56678
Number of validation data:  6298
Number of test data:  15744


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias', 'classifier.out

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/1772 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/1772 [00:00<?, ?it/s]

...

Running Epoch 1 of 3:   0%|          | 0/1772 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/1772 [00:00<?, ?it/s]

  0%|          | 0/15744 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/492 [00:00<?, ?it/s]

{'mcc': 0.7366542547372522, 'tp': 1883, 'tn': 12769, 'fp': 386, 'fn': 706, 'auroc': 0.9449922698714073, 'auprc': 0.8456107086276441, 'eval_loss': 0.2212786939831799}
Accuracy:  0.930640243902439
F1 score:  0.7752161383285302
------------------------------------------------------------


In [52]:
# grad_norm_result

In [53]:
count = 0
for result in grad_norm_result:
    print(f'----- gradient clipping {str(grad_norm_list[count])} -----')
    acc =  (result['tp']+result['tn'])/(result['tp']+result['tn']+result['fp']+result['fn'])
    precision = result['tp']/(result['tp']+result['fp']) 
    recall = result['tp']/(result['tp']+result['fn']) 
    f1 = (2*precision*recall)/(precision+recall)
    print('Accuracy: ', acc)
    print('Precision: ', precision)
    print('Recall: ', recall)
    print("F1 score: ", f1)
    print('\n')
    model_count += 1
    output_df.loc[model_count] = [32, 1, 4e-5, grad_norm_list[count], 'AdamW', acc, precision, recall, result['auroc'], f1]
    count += 1

----- gradient clipping 0.05 -----
Accuracy:  0.9317835365853658
Precision:  0.8362183754993342
Recall:  0.727694090382387
F1 score:  0.7781908302354399


----- gradient clipping 0.5 -----
Accuracy:  0.9298780487804879
Precision:  0.8199052132701422
Recall:  0.735032831208961
F1 score:  0.775152749490835


----- gradient clipping 1.5 -----
Accuracy:  0.930640243902439
Precision:  0.8298810048479507
Recall:  0.727307840865199
F1 score:  0.7752161383285302




In [54]:
output_df

,batch size,step,lr,grad_norm,optimizer,Accurcay,Precision,Recall,Auroc,F1 score
1,32,1,0.00004,1,AdamW,0.931974,0.849448,0.712630,0.941932,0.775047
2,32,1,0.00001,1,AdamW,0.930069,0.833333,0.718424,0.946809,0.771624
3,32,1,0.00002,1,AdamW,0.930767,0.834151,0.722673,0.945276,0.774421
4,32,1,0.00003,1,AdamW,0.930323,0.829505,0.725377,0.944827,0.773954
5,32,1,0.00005,1,AdamW,0.930577,0.829225,0.727694,0.943288,0.775149
6,8,1,0.00004,1,AdamW,0.919080,0.817479,0.653920,0.875916,0.726609
7,16,1,0.00004,1,AdamW,0.927973,0.828146,0.709154,0.941804,0.764045
8,32,2,0.00004,1,AdamW,0.931657,0.833407,0.730398,0.944792,0.778510
9,32,4,0.00004,1,AdamW,0.929116,0.816502,0.733874,0.946830,0.772986
10,32,1,0.00004,0.05,AdamW,0.931784,0.836218,0.727694,0.943632,0.778191


### 測試 optimizer

In [55]:
# opt_result = []
# opt_list = ["Adafactor"]

In [56]:
# for opt in opt_list:
#     banned_sample = chats[chats['delete']==True]
#     print('banned count:', len(banned_sample))
#     normal_sample = chats[chats['delete']==False].sample((5*len(banned_sample)),random_state=1)
#     print('sample normal count:', len(normal_sample))
#     sample_chats = normal_sample.append(banned_sample, ignore_index=True)

#     from sklearn.model_selection import train_test_split
#     X_train, X_test, y_train, y_test = train_test_split(sample_chats['body'], sample_chats['label'], test_size=0.2, random_state=42)
#     X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

#     print("Number of train data: ", len(X_train))
#     print("Number of validation data: ", len(X_valid))
#     print("Number of test data: ", len(X_test))
#     train_df = pd.concat([X_train, y_train], axis=1)
#     train_df.columns = ["text", "labels"]
#     valid_df = pd.concat([X_valid, y_valid], axis=1)
#     valid_df.columns = ["text", "labels"]
#     test_df = pd.concat([X_test, y_test], axis=1)
#     test_df.columns = ["text", "labels"]
#     train_df.head()


#     from simpletransformers.classification import ClassificationModel
#     output_name = "outputs/outputs_model_opt_"+opt+"/"
#     train_args = {
#         "output_dir": output_name,
#         "cache_dir": "cache/",
#         "overwrite_output_dir": True,

#         "fp16": False,
#         "fp16_opt_level": "O1",
#         "max_seq_length": 200,
#         "train_batch_size": 32,
#         "eval_batch_size": 32,
#         "gradient_accumulation_steps": 1,
#         "num_train_epochs": 3,
#         "weight_decay": 0,
#         "learning_rate": 4e-5,
#         "adam_epsilon": 1e-8,
#         "warmup_ratio": 0.06,
#         "warmup_steps": 0,
#         "max_grad_norm": 1.0,
#         "optimizer": opt,
#         "do_lower_case": False,
#         "save_model_every_epoch": False,
#         "reprocess_input_data": True,
#         "n_gpu": 1,
#         "silent": False,
#         "use_multiprocessing": False,
#         "use_early_stopping": True,
#         "early_stopping_patience": 3,
#         "early_stopping_delta": 0,
#         "early_stopping_metric": "eval_loss",
#         "early_stopping_metric_minimize": True,
#         "manual_seed": None,
#         "encoding": None,
#     }
#     model = ClassificationModel("roberta", "roberta-base", args=train_args)
#     model.train_model(train_df, eval_df=valid_df)

#     result, model_outputs, wrong_predictions = model.eval_model(test_df)
#     acc = (result['tp']+result['tn'])/(result['tp']+result['tn']+result['fp']+result['fn'])
#     precision = result['tp']/(result['tp']+result['fp']) 
#     recall = result['tp']/(result['tp']+result['fn'])  
#     f1 = (2*precision*recall)/(precision+recall)
#     print(result)
#     print("Accuracy: ", acc)
#     print("F1 score: ", f1)
#     print('-'*60)
#     opt_result.append(result) 

In [57]:
# opt_result

In [58]:
# count = 0
# for result in opt_result:
#     print(f'----- optimizer {str(opt_list[count])} -----')
#     acc =  (result['tp']+result['tn'])/(result['tp']+result['tn']+result['fp']+result['fn'])
#     precision = result['tp']/(result['tp']+result['fp']) 
#     recall = result['tp']/(result['tp']+result['fn']) 
#     f1 = (2*precision*recall)/(precision+recall)
#     print('Accuracy: ', acc)
#     print('Precision: ', precision)
#     print('Recall: ', recall)
#     print("F1 score: ", f1)
#     print('\n')
#     model_count += 1
#     output_df.loc[model_count] = [32, 1, 4e-5, 1, opt_list[count], acc, precision, recall, result['auroc'], f1]
#     count += 1

In [59]:
# output_df